In [1]:
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

#webscraping
#import requests
from bs4 import BeautifulSoup
import pickle
import time

In [2]:
'''
This has to be run only if the webscrapping script was run in batches!
Please skip if not needed and just load in single dataframe
'''
dir_dfs = !ls *df* #command line function to get all dfs in the dir
#use df_list to iterate over over all of the dfs 
df_list = []
for df in dir_dfs:
    df_list.append(df)
print(df_list)

#combines all of the dataframes together
dfs = []
for file in df_list:
    with open(file, 'rb') as picklefile: 
        df = pickle.load(picklefile)
        dfs.append(df)
df_main = pd.concat(dfs, ignore_index=True)
print("shape of final dataframe is", df_main.shape)

['final_df1.pkl', 'final_df10.pkl', 'final_df11.pkl', 'final_df12.pkl', 'final_df13.pkl', 'final_df14.pkl', 'final_df15.pkl', 'final_df16.pkl', 'final_df2.pkl', 'final_df3.pkl', 'final_df4.pkl', 'final_df5.pkl', 'final_df6.pkl', 'final_df7.pkl', 'final_df8.pkl', 'final_df9.pkl']


In [21]:
df_main.drop_duplicates(inplace=True)
df_main2 = df_main.copy()

In [15]:
print(df_main.head(2))

                                            Amenties Bathroom Bedrooms  \
0  <li>Fitness Center</li>\n<li>Barbeque Area</li...                     
1  <head> <meta charset="utf-8"/> <meta content="...                     

  Days_Posted Elevation  Landmarks  \
0          14     30 ft        9.0   
1                              0.0   

                                                Link  Neighborhood  \
0  https://www.trulia.com/c/ny/brooklyn/house-no-...  Williamsburg   
1  https://www.trulia.com/c/ny/brooklyn/the-green...                 

     Pets Allowed?                                               Rent Sqft  \
0  No pets allowed                                             $3,529   \n   
1                   harset="utf-8"/> <meta content="width=device-widt        

  Starbucks_dist                                              Title  \
0     0.22 miles  >House No.94 Apartments in Brooklyn, NY 11249 ...   
1                 >Access to this page has been denied.</title> ...   

  

In [67]:
#get rid of any sort of html strings "nan" and any listings with a range of rents
print(df_main2.shape)
df_main2 = df_main2[~(df_main2['Rent'].astype(str).str.contains(">"))]
df_main2 = df_main2[~(df_main2['Rent'].astype(str).str.contains("-"))]
df_main2 = df_main2[~(df_main2['Rent'].astype(str).str.contains("nan"))]
print(df_main2.shape)

#    big_df = big_df[~big_df['Rent'].str.contains('utf')] 
#df[(df['col_name'].str.contains('apple')) & (df['col_name'].str.contains('banana'))]

(1040, 14)
(843, 14)


In [22]:
def CleanMessyData2(big_df):
    '''
    Fix all of the KNOWN issues I encountered while scraping from Trulia 
    returns same dataframe with transformations made
    
    Input: Dataframe outputted from the GetMetrics Function 
    
    Returns: The same Dataframe with changes
    '''
    
    #removes all observations where we did not pick up proper rent values
    big_df = big_df[~(big_df["Rent"].astype(str).str.contains("nan", na=False))]
    big_df = big_df[~(big_df['Rent'].astype(str).str.contains(">", na=False))]
    big_df = big_df[~(big_df['Rent'].astype(str).str.contains("-", na=False))]

    #remove '$' and commas and convert to an int
    big_df['Rent'] = (big_df['Rent'].str.replace("$", "") \
                .str.replace(",","").
                astype(int))     
    
    #Extact bedroom values from the Title
    #Find bedroom extract digit, if none then 0 for studio
    for index, row in big_df.iterrows():
        row["Title"] = row["Title"].replace("Bedford", "Bdfrd")
        bed_index = row['Title'].find("Bed")
        #print("print row", str(index), "bed index", bed_index, "extr line", row['Title'][bed_index-3:bed_index-1])
        if bed_index == -1:
            big_df.set_value(index,'Bedrooms',0.0)
        else:
            temp = float(row["Title"][bed_index-3:bed_index-1])
            big_df.set_value(index,'Bedrooms',temp)
        
    #Extact bathroom values from the Title
    #Find bathroom extract digit, if none then 0
    for index, row in big_df.iterrows():
        row["Title"] = row["Title"].replace("Bath Ave", "Bth Ave")
        bath_index = row['Title'].find("Bath")
        #print("print row", str(index), "bath index", bath_index, "extr line", row['Title'][bath_index-3:bath_index-1])
        if bath_index == -1:
            big_df.set_value(index,'Bathroom',0.0)
        else:
            temp = float(row["Title"][bath_index-3:bath_index-1])
            big_df.set_value(index,'Bathroom',temp)

    
    #Map all kinds of pets to either pets allowed = Yes or No
    #Due to the logic only run this once 
    for index, row in big_df.iterrows():
        if "CAT" in row["Pets Allowed?"].upper():
            big_df.set_value(index,'Pets Allowed?', True)
        elif "DOG" in row["Pets Allowed?"].upper():
            big_df.set_value(index,'Pets Allowed?', True)
        else:
            big_df.set_value(index,'Pets Allowed?', False)

    #Remove % and sign from Tree_Cover_Pct feature
    big_df["Tree_Cover_Pct"] = (pd.to_numeric(big_df["Tree_Cover_Pct"]\
        .str.replace("%", "").str.strip(), errors='coerce'))
    #fill in NaNs with the neighborhood mean
    big_df['Tree_Cover_Pct'] = big_df['Tree_Cover_Pct']\
    .fillna(big_df.groupby('Neighborhood')['Tree_Cover_Pct'].transform('mean'))

    
    #Remove ft and strip out spaces from the Elevation Variable and convert to int
    #***Use mean to fill nans? > But do it with final dataframs 
    big_df["Elevation"] = (pd.to_numeric(big_df["Elevation"]\
        .str.replace("ft", "").str.strip(),errors='coerce'))
    #fill in NaNs with the neighborhood mean
    big_df['Elevation'] = big_df['Elevation']\
        .fillna(big_df.groupby('Neighborhood')['Elevation'].transform('mean'))
    
    #Remove ft and strip out spaces then convert to int
    #
    big_df["Starbucks_dist"] = (pd.to_numeric(big_df["Starbucks_dist"]\
        .str.replace("miles", "").str.strip(), errors='coerce'))
    #fill in NaNs with the neighborhood mean
    big_df['Starbucks_dist'] = big_df['Starbucks_dist']\
        .fillna(big_df.groupby('Neighborhood')['Starbucks_dist'].transform('mean'))

    
    big_df["Amenities_num"] = big_df["Amenties"].str.count("<li>") #counts the val of amenities
    for index, row in big_df.iterrows():
        if row["Amenities_num"] > 28:
            big_df.set_value(index,'Amenities_num',0.0)  # better number??
        else:
            continue
    big_df["Pets Allowed?"] = big_df["Pets Allowed?"].astype(int)
    big_df["Bedrooms"] = big_df["Bedrooms"].astype(float)
    big_df["Bathroom"] = big_df["Bathroom"].astype(float)
        
    big_df.reset_index(inplace=False) #reset since we removed rows
    return big_df

In [23]:
df_main2 = df_main.copy()
print("Size of dataframe before processing", df_main2.shape)
df_main2 = CleanMessyData2(df_main2)
print("Size of dataframe after processing", df_main2.shape)

(1474, 14)


/Users/danielobennett/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/Users/danielobennett/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:30: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/Users/danielobennett/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:45: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/Users/danielobennett/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:56: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


(1190, 15)


/Users/danielobennett/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:52: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/Users/danielobennett/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:54: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/Users/danielobennett/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:86: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


In [24]:
#drop all unnecessary columns cell
#df_main2.drop(["level_0", "index", "Title", "Link"])
df_main2.drop(["Title", "Link", "Amenties", "Sqft", "Days_Posted"], axis=1,inplace=True)
df_main2.columns

Index(['Bathroom', 'Bedrooms', 'Elevation', 'Landmarks', 'Neighborhood',
       'Pets Allowed?', 'Rent', 'Starbucks_dist', 'Tree_Cover_Pct',
       'Amenities_num'],
      dtype='object')

In [25]:
print(df_main2.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1190 entries, 0 to 1486
Data columns (total 10 columns):
Bathroom          1190 non-null float64
Bedrooms          1190 non-null float64
Elevation         1190 non-null float64
Landmarks         1190 non-null float64
Neighborhood      1190 non-null object
Pets Allowed?     1190 non-null int64
Rent              1190 non-null int64
Starbucks_dist    1190 non-null float64
Tree_Cover_Pct    1190 non-null float64
Amenities_num     1190 non-null int64
dtypes: float64(6), int64(3), object(1)
memory usage: 142.3+ KB
None


In [26]:
'''
FINAL STEP TO SAVE DATAFRAME AFTER CHANGES
'''
df_main2.reset_index(inplace=True, drop=True)
with open('data.pkl', 'wb') as picklefile:
    pickle.dump(df_main2, picklefile)